   # Multiband RF Pulse Design with PulPy

This notebook builds on the SLR pulse design notebook by multibanding a conventional SLR RF pulse for excitation in a simultaneous multislice imaging sequence. PINS multiband pulse design is also demonstrated.

In [ ]:
import numpy as np

import pulpy as pp
import pulpy.rf as rf

import sigpy.plot as pl
import scipy.signal as signal
import matplotlib.pyplot as pyplot

   ## Parameters for a time-bandwidth 4, linear-phase excitation

In [ ]:
tb = 8
N = 512
d1 = 0.01
d2 = 0.01
p_type = 'ex'
f_type = 'ls'

   ## Design the excitation pulse

In [ ]:
pulse = rf.slr.dzrf(N, tb, p_type, f_type, d1, d2, True)
pl.LinePlot(pulse, mode='r')

## Multiband it

In [ ]:
n_bands = 3
phs_type = 'phs_mod' # phsMod, ampMod, or quadMod
band_sep = 5*tb # separate by 5 slice widths
mb_pulse = rf.multiband.mb_rf(pulse, n_bands, band_sep, phs_type)
pl.LinePlot(mb_pulse)

## Simulate it

In [ ]:
[a, b] = pp.sim.abrm(mb_pulse, np.arange(-20*tb, 20*tb, 40*tb/2000), True)
Mxy = 2*np.multiply(np.conj(a), b)
pl.LinePlot(Mxy)

# Design a PINS pulse

In [ ]:
sl_sep = 3 # cm
sl_thick = 0.3 # cm
g_max = 4 # gauss/cm
g_slew = 18000 # gauss/cm/s
dt = 4e-6 # seconds, dwell time
b1_max = 0.18 # gauss
[rf_pins, g_pins] = rf.multiband.dz_pins(tb, sl_sep, sl_thick, g_max, g_slew, dt, b1_max, p_type, f_type, d1, d2)

In [ ]:
t = np.arange(0, np.size(rf_pins)) * dt
pyplot.figure()
pyplot.plot(t * 1000, np.real(rf_pins))
pyplot.ylabel('Gauss')
pyplot.xlabel('milliseconds')

In [ ]:
pyplot.figure()
pyplot.plot(t * 1000, g_pins)
pyplot.ylabel('Gauss/cm')
pyplot.xlabel('milliseconds')

In [ ]:
# simulate it
x = np.reshape(np.arange(-1000, 1000), (2000, 1)) / 1000 * 12 # cm
[a, b] = pp.sim.abrm_nd(2 * np.pi * dt * 4258 * rf_pins, x, 
                        np.reshape(g_pins, (np.size(g_pins), 1)) * 4258 * dt * 2 * np.pi)
Mxy = 2 * np.conj(a) * b

In [ ]:
pyplot.figure()
pyplot.plot(x, np.abs(Mxy))
pyplot.ylabel('|Mxy|')
pyplot.xlabel('cm')